# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,36.50,71,86,19.89,TF,1718293095
1,1,codrington,-38.2667,141.9667,44.40,78,100,8.57,AU,1718293095
2,2,lospalos,-8.5186,127.0025,67.96,84,24,4.38,TL,1718293095
3,3,waitangi,-43.9535,-176.5597,47.82,72,78,14.92,NZ,1718293095
4,4,arraial do cabo,-22.9661,-42.0278,84.06,42,0,17.27,BR,1718293095


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True,  
    tiles="EsriNatGeo",  
    size = "Humidity",
    scale = .50,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 65) & 
    (city_data_df["Max Temp"] <= 80) & 
    (city_data_df["Wind Speed"] < 4) & 
    (city_data_df["Humidity"] < 55)
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
163,163,inyonga,-6.7167,32.0667,71.62,45,0,3.60,TZ,1718293117
185,185,newman,37.3138,-121.0208,73.24,42,0,3.24,US,1718293119
246,246,dunda,-8.1167,34.2500,70.50,40,0,2.68,TZ,1718293127
335,335,vilaka,57.1846,27.6722,67.77,41,56,3.67,LV,1718293137
535,535,rexburg,43.8260,-111.7897,69.44,51,0,0.00,US,1718293165


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
163,inyonga,TZ,-6.7167,32.0667,45,
185,newman,US,37.3138,-121.0208,42,
246,dunda,TZ,-8.1167,34.2500,40,
335,vilaka,LV,57.1846,27.6722,41,
535,rexburg,US,43.8260,-111.7897,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Define the parameters for the API request
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "apiKey": geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Mikumi Lodge', 'country': 'Tanzania', 'country_code': 'tz', 'region': 'Western Zone', 'state': 'Katavi Region', 'county': 'Mlele', 'city': 'Inyonga', 'street': 'T23', 'lon': 32.056205, 'lat': -6.718725, 'formatted': 'Mikumi Lodge, T23, Inyonga, Katavi Region, Tanzania', 'address_line1': 'Mikumi Lodge', 'address_line2': 'T23, Inyonga, Katavi Region, Tanzania', 'categories': ['accommodation', 'accommodation.hotel', 'internet_access'], 'details': ['details.contact', 'details.facilities'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Mikumi Lodge', 'phone': '0742921261', 'osm_id': 5682249822, 'tourism': 'hotel', 'osm_type': 'n', 'internet_access': 'wlan'}}, 'contact': {'phone': '0742921261'}, 'facilities': {'internet_access': True}, '

,City,Country,Lat,Lng,Humidity,Hotel Name
163,inyonga,TZ,-6.7167,32.0667,45,Mikumi Lodge
185,newman,US,37.3138,-121.0208,42,No hotel found
246,dunda,TZ,-8.1167,34.2500,40,No hotel found
335,vilaka,LV,57.1846,27.6722,41,No hotel found
535,rexburg,US,43.8260,-111.7897,51,Quality Inn
551,connell,US,46.6635,-118.8611,41,No hotel found
559,laramie,US,41.3114,-105.5911,29,Laramie Valley Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True,  
    tiles="EsriNatGeo",  
    size="Humidity",
    scale=1.0,
    color="City",
    hover_cols=["City", "Country", "Hotel Name"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)